In [ ]:
%%bash
nrnivmodl

In [1]:
# Download NEURON: http://www.neuron.yale.edu/neuron/download
# Download PyNeuronToolbox: https://github.com/ahwillia/PyNeuron-Toolbox

from neuron import h
import numpy as np
import math
import pylab as plt
import scipy.linalg
from PyNeuronToolbox.record import ez_record,ez_convert
#from PyNeuronToolbox import neuromorpho,morphology
from PyNeuronToolbox.morphology import shapeplot,shapeplot_animate,load
from mpl_toolkits.mplot3d import Axes3D
from JSAnimation import IPython_display
from matplotlib import animation
from matplotlib.pyplot import cm
%matplotlib inline
np.random.seed(123456789)

import re
import pandas as pd

DC = 10 # diffusion coeff [microns^2 s^-1]


In [2]:
import sushibelt
import Edita_20reg_1dv_model as model
import time
N = model.N
tnorm = np.sum(model.target ** 2)
delta = 1e-5

2024-09-26 16:00:25,952 :: INFO :: function defined


	0 
total # of segments (50Hz): 742   max path distance: 853.32493 
apical 0  dend_5[0]
apical 1  dend_5[15]
apical 2  dend_5[14]
apical 3  dend_5[13]
apical 4  dend_5[12]
apical 5  dend_5[12]
apical 6  dend_5[12]
apical 7  dend_5[12]
apical 8  dend_5[12]
apical 9  dend_5[12]
apical 10  dend_5[12]
apical 11  dend_5[11]
apical 12  dend_5[11]
apical 13  dend_5[11]
apical 14  dend_5[10]
apical 15  dend_5[9]
apical 16  dend_5[8]
apical 17  dend_5[8]
apical 18  dend_5[8]
apical 19  dend_5[7]
apical 20  dend_5[6]
apical 21  dend_5[5]
apical 22  dend_5[5]
apical 23  dend_5[5]
apical 24  dend_5[5]
apical 25  dend_5[5]
apical 26  dend_5[5]
apical 27  dend_5[5]
apical 28  dend_5[4]
apical 29  dend_5[4]
apical 30  dend_5[4]
apical 31  dend_5[3]
apical 32  dend_5[2]
apical 33  dend_5[2]
apical 34  dend_5[2]
apical 35  dend_5[2]
apical 36  dend_5[2]
apical 37  dend_5[2]
apical 38  dend_5[2]
apical 39  dend_5[1]
apical 40  dend_5[1]
apical 41  dend_5[1]
#1


2024-09-26 16:00:26,024 :: INFO :: data read


In [3]:
#parval=pd.read_csv('../neuron-model/mprop_param.csv')
parval=pd.read_csv('../neuron-model/mprop_param_20240926.csv')
pval=parval[["F","Ctau","mProp","dvA","dvB","demand_CA1so_1","demand_CA1so_2","demand_CA1so_3","demand_CA1so_4","demand_CA1so_5","demand_CA1sr_1","demand_CA1sr_2","demand_CA1sr_3","demand_CA1sr_4","demand_CA1sr_5","demand_CA1sr_6","demand_CA1sr_7","demand_CA1sr_8","demand_CA1sr_9","demand_CA1sr_10","demand_CA1slm_1","demand_CA1slm_2","demand_CA1slm_3","demand_CA1slm_4","demand_CA1slm_5"]]
pval

,F,Ctau,mProp,dvA,dvB,demand_CA1so_1,demand_CA1so_2,demand_CA1so_3,demand_CA1so_4,demand_CA1so_5,...,demand_CA1sr_6,demand_CA1sr_7,demand_CA1sr_8,demand_CA1sr_9,demand_CA1sr_10,demand_CA1slm_1,demand_CA1slm_2,demand_CA1slm_3,demand_CA1slm_4,demand_CA1slm_5
0,0.000000e+00,-14.983069,0.00,-6.181901,-5.927975,6.204456e-01,3.772801e-01,0.317011,4.819997e-01,1.000000e+00,...,0.197723,0.101085,1.129671e-01,9.309864e-02,3.260768e-01,8.597611e-01,2.997236e-01,1.441719e-01,7.294553e-02,7.007312e-04
1,9.427898e-01,-5.259781,0.05,-5.945629,-17.998571,1.000001e-07,1.000007e-07,0.000282,1.000000e-07,1.001141e-07,...,0.053714,0.111154,1.240916e-01,2.475314e-01,1.000000e-07,1.001099e-07,1.000001e-07,1.107643e-01,2.156971e-01,1.000000e+00
2,1.000000e+00,-5.271068,0.10,-5.917387,-6.736725,1.000000e+00,1.000000e+00,0.906758,1.000323e-07,1.434509e-05,...,0.095073,0.133389,1.551426e-01,3.133888e-01,1.000002e-07,1.179863e-05,8.244570e-05,4.078140e-03,8.200315e-03,5.697236e-02
3,7.761776e-01,-5.296953,0.15,-5.865864,-18.000000,1.000000e-07,3.208574e-02,0.030430,1.370058e-02,1.000000e-07,...,0.124954,0.157183,1.821160e-01,3.426862e-01,3.278228e-01,1.000003e-07,1.310113e-01,1.704082e-01,2.588169e-01,1.000000e+00
4,9.999116e-01,-16.105336,0.20,-6.180935,-5.927161,6.180834e-01,3.753094e-01,0.315145,4.797774e-01,9.971657e-01,...,0.196037,0.099492,1.114819e-01,9.156652e-02,3.241917e-01,8.570113e-01,2.978329e-01,1.425597e-01,7.152941e-02,1.000000e-07
5,1.000000e+00,-5.329618,0.25,-5.784486,-17.996857,1.000000e+00,1.933616e-01,0.130092,1.020187e-01,1.000000e-07,...,0.319770,0.368132,4.300965e-01,7.953042e-01,1.000000e+00,1.000000e-07,8.387333e-02,1.400883e-01,2.206477e-01,1.000000e+00
6,1.000000e+00,-5.596598,0.30,-5.843128,-6.066726,1.000000e-07,1.000000e+00,0.670407,1.000000e+00,7.034418e-01,...,0.122661,0.148132,1.787957e-01,4.174542e-01,8.952121e-01,1.123722e-07,1.835290e-02,2.415132e-02,3.734382e-02,2.024113e-01
7,1.000000e+00,-6.075181,0.35,-5.859957,-6.464291,1.000000e+00,9.967107e-01,1.000000,1.043031e-07,2.389927e-04,...,0.257936,0.326926,3.784057e-01,9.073133e-01,9.999618e-01,1.075679e-07,2.374624e-02,4.680033e-02,7.926509e-02,4.323790e-01
8,9.794017e-01,-5.894524,0.40,-5.736678,-11.423467,2.131350e-01,6.075718e-02,0.047838,4.554650e-02,1.000000e-07,...,0.155489,0.173367,2.032740e-01,3.740324e-01,5.235525e-01,4.279650e-02,1.963693e-01,1.957351e-01,2.753806e-01,1.000000e+00
9,6.805197e-01,-17.999626,0.45,-6.180792,-5.928607,6.200608e-01,3.762692e-01,0.316038,4.812507e-01,1.000000e+00,...,0.196559,0.099697,1.116483e-01,9.174532e-02,3.250310e-01,8.594977e-01,2.985816e-01,1.429248e-01,7.161642e-02,1.000000e-07


In [4]:
aL=list()
bL=list()
dL=list()
cL=list()
for i in range(pval.shape[0]):
    parr=np.array(pval.iloc[i])
    F = parr[0]
    Ctau = 10 ** parr[1]
    mProp = parr[2]
    dvA = parr[3]
    dvB = parr[4]
    dv = np.zeros(N)
    utarg = delta*np.ones(N)
    for k in range(N):
        if model.itarg[k] > 4:
            utarg[k] = parr[model.itarg[k]]
            dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = model.trafficking_solution(F * utarg + (1 - F) * K)
    x=np.append(x,0)
    a = (1 / (1 + x))
    aM, aF = sushibelt.aggregate_segments(np.concatenate((list(a),list(a))), model.segIdx, model.expD['Abbreviation'], fun=np.sum)
    aL.append(aM)
    b = list((1 / (1 + x ** -1)))
    bM, bF = sushibelt.aggregate_segments(np.concatenate((list(b),list(b))), model.segIdx, model.expD['Abbreviation'], fun=np.sum)
    bL.append(bM)
    dM, dF = sushibelt.aggregate_segments(np.concatenate((list(dv),list(dv))), model.segIdx, model.expD['Abbreviation'], fun=np.sum)
    dL.append(dM)
    c = Ctau * utarg / (F * utarg + (1 - F) * K)
    cM, cF = sushibelt.aggregate_segments(np.concatenate((list(c),list(c))), model.segIdx, model.expD['Abbreviation'], fun=np.sum)
    cL.append(cM)
(len(aL),len(bL),len(cL),len(dL),len(aL[0]))

/var/folders/z6/t85_f56n6kvg57rq9jhcxj480000gn/T/ipykernel_30855/1534176074.py:25: RuntimeWarning: divide by zero encountered in reciprocal
  b = list((1 / (1 + x ** -1)))


(21, 21, 21, 21, 20)

In [5]:
adf = pd.DataFrame(aL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
bdf = pd.DataFrame(bL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
cdf = pd.DataFrame(cL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
ddf = pd.DataFrame(dL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
#adf.to_csv('a_mprop.csv')
#bdf.to_csv('b_mprop.csv')
#cdf.to_csv('c_mprop.csv')
#ddf.to_csv('d_mprop.csv')
#####
adf.to_csv('a_mprop_20240926.csv')
bdf.to_csv('b_mprop_20240926.csv')
cdf.to_csv('c_mprop_20240926.csv')
ddf.to_csv('d_mprop_20240926.csv')

In [6]:
aL=list()
bL=list()
dL=list()
cL=list()
for i in range(pval.shape[0]):
    parr=np.array(pval.iloc[i])
    F = parr[0]
    Ctau = 10 ** parr[1]
    mProp = parr[2]
    dvA = parr[3]
    dvB = parr[4]
    dv = np.zeros(N)
    utarg = delta*np.ones(N)
    for k in range(N):
        if model.itarg[k] > 4:
            utarg[k] = parr[model.itarg[k]]
            dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = model.trafficking_solution(F * utarg + (1 - F) * K)
    x=np.append(x,0)
    a = (1 / (1 + x))
    aM, aF = sushibelt.aggregate_segments(np.concatenate((list(a),list(a))), model.segIdx, model.expD['Abbreviation'], fun=np.mean)
    aL.append(aM)
    b = list((1 / (1 + x ** -1)))
    bM, bF = sushibelt.aggregate_segments(np.concatenate((list(b),list(b))), model.segIdx, model.expD['Abbreviation'], fun=np.mean)
    bL.append(bM)
    dM, dF = sushibelt.aggregate_segments(np.concatenate((list(dv),list(dv))), model.segIdx, model.expD['Abbreviation'], fun=np.mean)
    dL.append(dM)
    c = Ctau * utarg / (F * utarg + (1 - F) * K)
    cM, cF = sushibelt.aggregate_segments(np.concatenate((list(c),list(c))), model.segIdx, model.expD['Abbreviation'], fun=np.mean)
    cL.append(cM)
(len(aL),len(bL),len(cL),len(dL),len(aL[0]))

/var/folders/z6/t85_f56n6kvg57rq9jhcxj480000gn/T/ipykernel_30855/893620433.py:25: RuntimeWarning: divide by zero encountered in reciprocal
  b = list((1 / (1 + x ** -1)))


(21, 21, 21, 21, 20)

In [7]:
adf = pd.DataFrame(aL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
bdf = pd.DataFrame(bL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
cdf = pd.DataFrame(cL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
ddf = pd.DataFrame(dL,columns=model.expD['Abbreviation'],index=[pval.iloc[k,2] for k in range(pval.shape[0])])
#adf.to_csv('a_mprop.mean.csv')
#bdf.to_csv('b_mprop.mean.csv')
#cdf.to_csv('c_mprop.mean.csv')
#ddf.to_csv('d_mprop.mean.csv')
######
adf.to_csv('a_mprop.mean_20240926.csv')
bdf.to_csv('b_mprop.mean_20240926.csv')
cdf.to_csv('c_mprop.mean_20240926.csv')
ddf.to_csv('d_mprop.mean_20240926.csv')

In [10]:
(len(aL),len(model.parnames))

(21, 25)

In [9]:
aM, aF = sushibelt.aggregate_segments(np.concatenate((aL[0],aL[0])), model.segIdx, model.expD['Abbreviation'], fun=np.sum)

IndexError: index 651 is out of bounds for axis 0 with size 40

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,aM,label='a')
#ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,model.expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title("A[0]") 
# function to show the plot
plt.show()
